In [1]:
import optuna
import pandas as pd
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import train_test_split

from project_ml_course.data_process import filter_columns_by_correlation_threshold

/Users/vitor.mello/vho/project_ml_course/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
raw_df = pd.read_csv("../data/dados.csv", index_col="Unnamed: 0")

df = filter_columns_by_correlation_threshold(
    df=raw_df,
    ref_col="class",
    method_type="pearson",
    lower_threshold=0.001,
    higher_threshold=0.999,
)

X = df.drop(columns=["class"])
y = df["class"]

# Separar treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [3]:
def objective(trial):
    C = trial.suggest_loguniform("C", 1e-3, 100)
    kernel = trial.suggest_categorical("kernel", ["rbf", "linear", "poly", "sigmoid"])
    # Parâmetros adicionais para poly
    degree = 3
    if kernel == "poly":
        degree = trial.suggest_int("degree", 2, 5)
    svm = SVC(
        C=C,
        kernel=kernel,
        degree=degree,
        probability=True,
        class_weight="balanced",
        random_state=42,
    )
    pipe = Pipeline(
        [
            ("scaler", StandardScaler(with_std=False)),
            ("pca", PCA(n_components=3, random_state=42)),
            ("svm", svm),
        ]
    )
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(
        pipe, X_train, y_train, cv=cv, scoring=make_scorer(f1_score)
    )
    return scores.mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("Melhores hiperparâmetros encontrados:")
print(study.best_params)
print(f"Melhor F1 médio (CV): {study.best_value:.4f}")


[I 2025-07-15 21:48:00,712] A new study created in memory with name: no-name-f55bfbf8-4d55-4920-ac0e-74df5da97103
  0%|          | 0/30 [00:00<?, ?it/s]/var/folders/p5/0wjpw3594x53gr6w8yv_trj40000gq/T/ipykernel_37394/190903671.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 100)
Best trial: 0. Best value: 0.574879:   3%|▎         | 1/30 [00:00<00:12,  2.36it/s]/var/folders/p5/0wjpw3594x53gr6w8yv_trj40000gq/T/ipykernel_37394/190903671.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 100)


[I 2025-07-15 21:48:01,142] Trial 0 finished with value: 0.5748794307449971 and parameters: {'C': 0.01707661096569344, 'kernel': 'poly', 'degree': 3}. Best is trial 0 with value: 0.5748794307449971.


Best trial: 0. Best value: 0.574879:   7%|▋         | 2/30 [00:00<00:11,  2.48it/s]/var/folders/p5/0wjpw3594x53gr6w8yv_trj40000gq/T/ipykernel_37394/190903671.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 100)


[I 2025-07-15 21:48:01,532] Trial 1 finished with value: 0.4377400423258053 and parameters: {'C': 24.582642407691132, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.5748794307449971.


Best trial: 0. Best value: 0.574879:  10%|█         | 3/30 [00:27<05:35, 12.43s/it]/var/folders/p5/0wjpw3594x53gr6w8yv_trj40000gq/T/ipykernel_37394/190903671.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 100)


...

[I 2025-07-15 21:49:04,944] Trial 27 finished with value: 0.798942512038718 and parameters: {'C': 26.404956329062642, 'kernel': 'poly', 'degree': 2}. Best is trial 16 with value: 0.9784599917708607.


Best trial: 16. Best value: 0.97846:  97%|█████████▋| 29/30 [01:04<00:00,  3.06it/s]/var/folders/p5/0wjpw3594x53gr6w8yv_trj40000gq/T/ipykernel_37394/190903671.py:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 1e-3, 100)


[I 2025-07-15 21:49:05,193] Trial 28 finished with value: 0.9695565633254255 and parameters: {'C': 2.4009748448419623, 'kernel': 'rbf'}. Best is trial 16 with value: 0.9784599917708607.


Best trial: 16. Best value: 0.97846: 100%|██████████| 30/30 [01:04<00:00,  2.17s/it]

[I 2025-07-15 21:49:05,669] Trial 29 finished with value: 0.468012329996804 and parameters: {'C': 0.44706323454078883, 'kernel': 'linear'}. Best is trial 16 with value: 0.9784599917708607.
Melhores hiperparâmetros encontrados:
{'C': 10.251516094582083, 'kernel': 'rbf'}
Melhor F1 médio (CV): 0.9785
